In [1]:
%config IPCompleter.greedy=True

import os
import glob
import cv2 as cv 
import numpy as np
import math
import matplotlib
import PIL
import matplotlib.pyplot as plt
from PIL import Image, ImageEnhance
from math import sqrt

dirname = os.path.abspath('')

class Line:
    """
    Store a line based on the two points.
    """
    def __init__(self, point_1, point_2):
        self.point_1 = point_1
        self.point_2 = point_2
        
    """
    Return the two points that generate the line
    """
    def get_points(self):
        return self.point_1, self.point_2
        
    """
    Return the scalars that explain the line
    Ax + By + C = 0
    """ 
    def get_line_coeff(self):
        A = self.point_1.y - self.point_2.y
        B = self.point_2.x - self.point_1.x
        C = self.point_1.x * self.point_2.y - self.point_2.x * self.point_1.y
        return A,B,C
    
    """
    Generate new points to extend an existing line
    It is used to extend the lines generated by the Hough transform
    over the limits of the Sudoku Grid
    """
    def extend_on_xaxis(self, x):
        A,B,C = self.get_line_coeff()
        new_y = -(A*x+C)/B
        return new_y
    
    def extend_on_yaxis(self, y):
        A,B,C = self.get_line_coeff()
        new_x = -(B*y+C)/A
        return new_x

class Point:
    """
    Class used to describe a point with its coordinates
    """
    def __init__(self, x, y):
        self.x = np.int32(np.round(x))
        self.y = np.int32(np.round(y))
    
    def get_point_as_tuple(self):
        return (self.x, self.y)
    
class Patch:
    """
    Store information about each item  in the table. 
    """
    def __init__(self, image_patch, x_min, y_min, x_max, y_max, line_idx, column_idx):
        self.image_patch = image_patch
        self.x_min = x_min
        self.y_min = y_min
        self.x_max = x_max
        self.y_max = y_max
        self.line_idx = line_idx
        self.column_idx = column_idx
        self.has_x: int = 0 # 0 meaning it does not contain an 'X', 1 meaning it contains an 'X'
    
    def set_x(self, has_x: int):
        assert has_x == 0 or has_x == 1 # convention 
        self.has_x = has_x

        
def list_of_files(dir_name, extension="jpg"):
    """
    Helper function that returns a list of all 
    files in a directory with a specific extension
    :param dir_name.
    :param extension.
    :return List of all files with that specific extension
    """
    return [f for f in glob.glob(dir_name+"*." + extension)]


def show_image(image, window_name='image'):
    """
    Helper function used to show images
    """
    plt.figure(figsize=(12,6))
    plt.title(window_name)
    plt.imshow(image, cmap='gray')
    
    
def draw_lines(image, lines: [Line], color: tuple = (0, 0, 255),
           return_drawing: bool = False, window_name: str = 'window'):
    """
    Plots the lines into the image.
    :param image.
    :param lines.
    :param color. The color used to draw the lines
    :param return_drawing. Use it if you want the drawing to be return instead of displayed.
    :return None if return_drawing is False, otherwise returns the drawing.
    """
    drawing = image.copy()
    if drawing.ndim == 2:
        drawing = cv.cvtColor(drawing, cv.COLOR_GRAY2BGR)
    for line in lines: 
        cv.line(drawing, line.point_1.get_point_as_tuple(), line.point_2.get_point_as_tuple(), color, 2, cv.LINE_AA)

    if return_drawing:
        return drawing
    else:
        show_image(drawing, window_name=window_name)


def remove_close_lines(lines: [Line], threshold: int, is_vertical: bool):
    """
    It removes the closest lines.
    :param lines.
    :param threshold. It specify when the lines are too close to each other.
    :param is_vertical. Set it to True or False.
    :return : The different lines.
    """
    
    different_lines = [] 
    if is_vertical:
        lines.sort(key=lambda line: line.point_1.x)
    else:
        lines.sort(key=lambda line: line.point_1.y)
    
    #  add the first line
    different_lines.append(lines[0])
    if is_vertical:
        for line_idx in range(1, len(lines)):
            if lines[line_idx].point_1.x - different_lines[-1].point_1.x > threshold:
                different_lines.append(lines[line_idx])
    else:
        for line_idx in range(1, len(lines)): 
            if lines[line_idx].point_1.y - different_lines[-1].point_1.y > threshold:
                different_lines.append(lines[line_idx])
    return different_lines
 

def slope(x1, y1, x2, y2):
    """
    Calculate the slope of a line by having 
    the coordinates of two points that describe the line
    """
    m = (y2-y1)/(x2-x1)
    return m


def get_horizontal_and_vertical_lines_hough(edges, threshold=150, minLineLength=80, maxLineGap=5):
    """
    Returns the horizontal and vertical lines found by Hough transform.
    :param edges = the edges of the image.
    :threshold = it specifies how many votes need a line to be considered a line.
    :minLineLength = minimum length of a line 
    :maxLineGap 
    :return (horizontal_lines: List(Line), vertical_lines: List(Line))
    """
    
    lines = cv.HoughLinesP(image=edges,rho=1.0,theta=np.pi/180, threshold=threshold,lines=np.array([]), 
                           minLineLength=minLineLength,maxLineGap=maxLineGap)
    lines = np.array(lines)

    vertical_lines: [Line] = []
    horizontal_lines: [Line] = [] 
    
    for i in range(0, len(lines)):
        # TODO: compute the line coordinate
        
        m = slope(lines[i][0][0], lines[i][0][1], lines[i][0][2], lines[i][0][3])
        
        pt1 = [lines[i][0][0], lines[i][0][1]]
        pt2 = [lines[i][0][2], lines[i][0][3]]
        
        line = Line(Point(x=pt1[0], y=pt1[1]), Point(x=pt2[0], y=pt2[1])) 
        
        if -0.5 <= m <= 0.5:
            new_y1 = line.extend_on_xaxis(x=0)
            new_y2 = line.extend_on_xaxis(x=400)
            new_line = Line(Point(x=0, y=new_y1), Point(x=400, y=new_y2)) 
            
            horizontal_lines.append(new_line)
        elif -1.5 >= m or 1.5 <= m:
            new_x1 = line.extend_on_yaxis(y=0)
            new_x2 = line.extend_on_yaxis(y=400)
            new_line = Line(Point(x=new_x1, y=0), Point(x=new_x2, y=400)) 
            
            vertical_lines.append(new_line) 
    
    return horizontal_lines, vertical_lines


def get_intersect(line1, line2):
    """ 
    Returns the point of intersection of the lines passing through a2,a1 and b2,b1.
    a1: [x, y] a point on the line1
    a2: [x, y] another point on the line1
    b1: [x, y] a point on the line2
    b2: [x, y] another point on the line2
    """
    a1, a2 = line1.get_points()
    b1, b2 = line2.get_points()
    s = np.vstack([a1.get_point_as_tuple(),a2.get_point_as_tuple()
                   ,b1.get_point_as_tuple(),b2.get_point_as_tuple()])        # s for stacked
    h = np.hstack((s, np.ones((4, 1)))) # h for homogeneous
    l1 = np.cross(h[0], h[1])           # get first line
    l2 = np.cross(h[2], h[3])           # get second line
    x, y, z = np.cross(l1, l2)          # point of intersection
    if z == 0:                          # lines are parallel
        return (float('inf'), float('inf'))
    return [round(x/z), round(y/z)]


class MagicClassifier:
    """
    A very strong classifier that detects if the patch has an 'X' or not.
    """
    def __init__(self):
        self.threshold = 20
    
    def classic_classify(self, patch: Patch) -> int:
        """
        Receive a Patch and return 1 if there is an 'X' in the patch and 0 otherwise.
        """ 
        if np.std(patch.image_patch) < self.threshold:
            return 0
        else: 
            return 1

def classify_patches_with_magic_classifier(patches: [Patch]) -> None:
    """
    Receive the patches and classify if the patch contains an 'X' or not.
    :param patches.
    :return None
    """
    magic_classifier = MagicClassifier()
    for patch in patches:
        patch.set_x(magic_classifier.classic_classify(patch))

        
def show_patches_which_have_x(patches: [Patch]) -> None:
    """
    This function draws a colored rectangle if the patch has an 'X'. 
    """
    image_color = np.zeros((400, 400, 3), np.uint8)
    y_min = patches[0].y_min
    for patch in patches:
        x_min_current = patch.x_min 
        y_min_current = patch.y_min 
        x_max_current = patch.x_max 
        y_max_current = patch.y_max 

        image_color[y_min_current: y_max_current, x_min_current: x_max_current] = np.dstack((patch.image_patch, patch.image_patch, patch.image_patch))
        
        if patch.has_x == 1:  
            cv.rectangle(image_color, (x_min_current, y_min_current), 
                         (x_max_current, y_max_current), color=(255, 0, 0), thickness=2)
    
    #show_image(image_color)
    
def return_patches(patches: [Patch]):
    """
    Return a 9x9 matrix with 'x' and 'o' that describe 
    if a patch has an x or not in it.
    """
    results = np.full([81], None)
    for i, patch in enumerate(patches):
        if patch.has_x == 1:  
            results[i] = 'x'
        else:
            results[i] = 'o'
    results = results.reshape(9,9)    
    return results

opencv_python==3.4.2
numpy==1.19.2
matplotlib==3.3.2
PIL==8.0.1


In [2]:
def get_patches(lines, columns, image, step=8, show_patches = False):
    """
    It cuts out each box from the table defined by the lines and columns.
    :param lines. The lines that difine the table.
    :param columns. The columns that difine the table.
    :param image. The image containing the table.
    :param show_patches. Determine if the patches will be drawn on the image or not.
    :param step. The number of pixels that represent the padding between the actual
                extracted patch and the borders given by the sudoku lines
    :return : A list with all boxes in the table.
    """
    
    def crop_patch(image_, x_min, y_min, x_max, y_max, crop_points):
        """
        Crops the bounding box represented by the coordinates.
        """
        b_box = image_[y_min: y_max, x_min: x_max].copy()
        
        mask = np.ones_like(b_box) * 255
        
        res = cv.bitwise_and(b_box,b_box,mask = mask)

        return res
    
    def draw_patch(image_, patch: Patch, points, color: tuple = (255, 0, 255)):
        """
        Draw the irregular shape corresponding to the patch on the image.
        """
        cv.polylines(image_,[points],True,color)
    
    if len(image.shape) == 3:
        image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    if show_patches: 
        image_color = np.dstack((image, image, image))
  
    lines.sort(key=lambda line: line.point_1.y)
    columns.sort(key=lambda column: column.point_1.x)
    patches = []
    step = step
    for line_idx in range(len(lines) - 1):
        for col_idx in range(len(columns) - 1):
            
            # get the 4 points of intersection between two consecutive
            # vertical and horizontal lines
            p1 = get_intersect(lines[line_idx], columns[col_idx])
            p2 = get_intersect(lines[line_idx], columns[col_idx+1])
            p3 = get_intersect(lines[line_idx+1], columns[col_idx+1])
            p4 = get_intersect(lines[line_idx+1], columns[col_idx])
            
            points = np.array([p1,p2,p3,p4])
            
            # define the limits of the irregular shape
            x_min = np.min(points[:,0]) + step
            x_max = np.max(points[:,0]) - step
            
            y_min = np.min(points[:,1]) + step
            y_max = np.max(points[:,1]) - step
            
            patch = Patch(image_patch=crop_patch(image,  x_min, y_min, x_max, y_max, points),
                          x_min=x_min, y_min=y_min, x_max=x_max, y_max=y_max,
                          line_idx=line_idx, column_idx=col_idx)
            
            if show_patches:
                draw_patch(image_color, patch, points)
            
            patches.append(patch)

            
    if show_patches:
        show_image(image_color, window_name='Determined patches')
    return patches

In [3]:
def change_perspective(img_path, brightness_factor=1.2, 
                       fx=0.125, fy=0.125):
    """
    Change the perspective of the initial image to a new one with the axis aligned
    :param img_path. The path to the image that needs to be warped
    :param brightness_factor. An enhancing of the brightness is done in order to remove 
                            a part of the color noise
    :param fx. Define the ratio at which the image will be resized on the X axis
    :param fy. Define the ratio at which the image will be resized on the Y axis
    :return : The warped image
    """
    # open the image in RGB (specific to PIL open process)
    pil_image = Image.open(img_path)

    # apply the brightness increasing
    enhancer = ImageEnhance.Brightness(pil_image)
    factor = brightness_factor
    pil_image = enhancer.enhance(factor)

    # convert the image from RGB to BGR and resize it
    sample_image = cv.cvtColor(np.array(pil_image), cv.COLOR_RGB2BGR)

    img_shape = sample_image.shape

    sample_image = cv.resize(sample_image,None,fx=fx,fy=fy)

    # convert the image to grayscale and apply an adaptive threshold to binarize 
    # the important features in the images
    gray_image = cv.cvtColor(sample_image, cv.COLOR_BGR2GRAY)
    blur = cv.medianBlur(gray_image, 3)
    edges = cv.adaptiveThreshold(blur,255,cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY_INV,11,3)

    # find the contours and take the biggest one as a reference
    _, contours, _ = cv.findContours(edges,cv.RETR_TREE,cv.CHAIN_APPROX_SIMPLE)
    cnt = sorted(contours, key=cv.contourArea, reverse=True)
    biggest_cont = cnt[0]

    # create a mask using the biggest contour
    mask = np.zeros_like(gray_image)
    cv.drawContours(mask,[biggest_cont],0,255,-1)
    cv.drawContours(mask,[biggest_cont],0,0,2)
    res = cv.bitwise_and(sample_image,sample_image,mask = mask)

    # get the rectangle that best describes the biggest contour and 
    # take the four points of the rectangle
    biggest_cont = np.squeeze(biggest_cont, axis=1)
    box_corners = []
    rot_rect = cv.minAreaRect(biggest_cont)
    box = cv.boxPoints(rot_rect)
    box = np.int0(box)
    for p in box:
        box_corners.append([p[0],p[1]])

    # check the rotation of the sudoku grid
    # it is important to understand the orientation of the sudoku grid
    # in order to have a proper perspective warping
    if box_corners[1][1] > box_corners[3][1]:
        dst = [[350,350],[50,350],[50,50],[350,50]]
    else:
        dst = [[50,350],[50,50],[350,50],[350,350]]

    # calculate the transform matrix needed for the warping process
    M = cv.getPerspectiveTransform(np.float32(box_corners), np.float32(dst))

    # warp the initial image using the M transform matrix
    warped = cv.warpPerspective(res, M, res.shape[:2], flags=cv.INTER_LINEAR)
    
    return warped, M

In [4]:
def predict_patches(warped, task_no, step=8, hough_thresh=100,
                   minLineLength=80, maxLineGap=5, show_results=False):
    """
    Detect the lines corresponding to the sudoku grid and based on them
    draw and determine the important features of the patches
    :param warped. The warped image as the input
    :param task_no. 1 or 2 based on the task that the function solves
    :param step. Parameter used for the get_patches function (see get_patches)
    :param hough_thresh. Parameter used for get_horizontal_and_vertical_lines_hough function 
    :param minLineLength. Parameter used for get_horizontal_and_vertical_lines_hough function 
    :param maxLineGap. Parameter used for get_horizontal_and_vertical_lines_hough function 
    :param show_results. Print the images
    :return : The matrix of the presence of features in patches, and for task 2, also return the patches
    """
    
    edges = cv.Canny(warped,90,255)
    
    if show_results:
        show_image(edges, "Canny edges detection")
        
    if task_no == 1:
        kernel = np.ones((2,2),np.uint8)
        edges = cv.dilate(edges,kernel)
        
    if task_no == 2:
        kernel = np.ones((3,3),np.uint8)
        edges = cv.dilate(edges,kernel)

    if show_results:
        show_image(edges, "Canny after morphological dilation procedure")
    
    # get the horizontal and vertical lines based on hough detection
    horizontal_lines, vertical_lines = get_horizontal_and_vertical_lines_hough(edges,hough_thresh,minLineLength,maxLineGap)

    if show_results:
        draw_lines(warped, horizontal_lines, window_name='Horizontal lines')
        draw_lines(warped, vertical_lines, window_name='Vertical lines')

    distinct_horizontal_lines = remove_close_lines(horizontal_lines, threshold = 20, is_vertical = False)
    distinct_vertical_lines = remove_close_lines(vertical_lines, threshold = 20, is_vertical = True)

    if show_results:
        draw_lines(warped, distinct_vertical_lines, window_name='Vertical lines after removing')
        draw_lines(warped, distinct_horizontal_lines, window_name='Horizontal lines after removing')

    patches = get_patches(distinct_horizontal_lines, distinct_vertical_lines, warped, step,
                                  show_patches=False)

    classify_patches_with_magic_classifier(patches) 

    if task_no == 1:
        show_patches_which_have_x(patches)
        return return_patches(patches)
    if task_no == 2:
        show_patches_which_have_x(patches)
        return return_patches(patches), patches  

In [5]:
def evaluate_task1(train_or_test='train'):
    """
    Evaluate task 1 on the 'classic' input set
    :param train_or_test. The source of the files. It can be either 'train' or 'test'.
            To find the input images, the root folder where the notebook is located is taken
            into consideration.
    :output. The files with the resulting output will be saved at:
            <current_directory>/evaluation/submission_files/classic/"
    """
    print("Evaluate task 1")
    
    # get the path to the images and define all the other necessary paths
    imgs_path = list_of_files(dirname + "/" + train_or_test + "/classic/")
    predictions_path_root = "/evaluation/submission_files/"
    predictions_path = dirname + predictions_path_root + "classic/"

    for i in range(len(imgs_path)):
        print("Image " + str(i+1) + " from " + str(len(imgs_path)) + " for task 1")
        warped, M = change_perspective(imgs_path[i])
        result = predict_patches(warped, task_no=1, show_results=False)
        img_no = imgs_path[i].split("/")[-1].split(".")[0]
        
        # save the results, line by line, to *_predicted files
        with open(predictions_path + str(img_no) + '_predicted.txt','w') as f:
            for j, line in enumerate(result):
                line_str = ''.join(line)
                if j < 8:
                    f.write(line_str + "\n")
                else:
                    f.write(line_str)

In [6]:
def build_gabor_filters(theta_list, lambda_list) -> list:
    """
    Build Gabor filters with the window size of 55 pixels and theta and lambda received in the paramters.
    :param theta_list. The list with the theta parameters.
    :param lambda_list. The list the lambda paramters.
    :return. list of Gabor filters.
    """
    filters = []
    ksize = 55
    
    for lambda_ in lambda_list:   
        for theta in theta_list:    
            kern = cv.getGaborKernel((ksize, ksize), 4, theta, lambda_, 0.5, 0, ktype=cv.CV_32F)
            kern /= 1.5 * kern.sum()
            filters.append(kern)            
    return filters

def apply_filters(img, filter):
    """
    :param img - the image on which we apply the filters.
    :param filter - a list with filters.
    :return. The results of applying the filters on the image.
    """ 
    response = np.abs(cv.filter2D(img, cv.CV_32F, filter))
    return (255*(response - np.min(response))/np.ptp(response)).astype(int) 
        
def dfs(horizontal, vertical, ids, current_i, current_j, current_id):
    """
    A depth first search to determine the patches that correspond to 
    the same jigsaw sudoku region
    :param horizontal - a 8x8 matrix that contains all the horizontal thick lines in the irregular sudoku grid
    :param vertical - a 8x8 matrix that contains all the vertical thick lines in the irregular sudoku grid
    :param ids - a 9x9 matrix that contains the ids of each of the patches from the sudoku grid
    :param current_i - the current i position
    :param current_j - the current j position
    :param current_id - the current id of the i,j position
    :return. The 9x9 matrix that contains all the ids 
    """ 
    if ids[current_i,current_j] != 0:
        return

    ids[current_i,current_j] = current_id

    neigh = []

    if current_i > 0:
        # north
        if horizontal[current_i-1,current_j] == 0:
            neigh.append([current_i-1,current_j])

    if current_i < 8:
        # south
        if horizontal[current_i,current_j] == 0:
            neigh.append([current_i+1,current_j])

    if current_j > 0:
        # west
        if vertical[current_i,current_j-1] == 0:
            neigh.append([current_i,current_j-1])

    if current_j < 8:
        # east
        if vertical[current_i,current_j] == 0:
            neigh.append([current_i,current_j+1])

    for next_i, next_j in neigh:
        dfs(horizontal, vertical, ids, next_i, next_j, current_id)

In [7]:
def evaluate_task2(train_or_test='train'):
    """
    Evaluate task 2 on the 'jigsaw' input set
    :param train_or_test. The source of the files. It can be either 'train' or 'test'.
            To find the input images, the root folder where the notebook is located is taken
            into consideration.
    :output. The files with the resulting output will be saved at:
            <current_directory>/evaluation/submission_files/jigsaw/"
    """
    print("Evaluate task 2")
    imgs_path = list_of_files(dirname+"/" + train_or_test + "/jigsaw/")
    
    predictions_path_root = "/evaluation/submission_files/"

    predictions_path = dirname + predictions_path_root + "jigsaw/"

    # define the theta list as 0 and pi/2 because we want to determine the horizontal and vertical thick lines
    theta_list = [0, np.pi/2]
    # the lambda parameter list was defined as [8,10]
    lambda_list = [8, 10]
    gabor_filters = build_gabor_filters(theta_list=theta_list, lambda_list=lambda_list)

    for i in range(len(imgs_path)):
        print("Image " + str(i+1) + " from " + str(len(imgs_path)) + " for task 2")
        warped, M = change_perspective(imgs_path[i])
        result, patches = predict_patches(warped, task_no=2, show_results=False)
        
        mask = np.zeros(warped.shape[:2])

        # apply the filters over the resulted warped image
        warped = cv.cvtColor(warped,cv.COLOR_BGR2GRAY)
        for gabor_filter in gabor_filters[2:]:
            response = apply_filters(warped, gabor_filter)
            mask[response>150] = 255

        horizontal_thick_lines = np.zeros(81)
        vertical_thick_lines = np.zeros((81))
        
        # check if the region between two patches (either vertical or horizontal) 
        # contains any "white" points that corresponds to the thick lines
        for j in range(len(patches)-1):
            # check the horizontal regions between patches
            if (j == 0) or ((j+1) % 9 != 0):
                y_mean = round((patches[j].y_max+patches[j].y_min)/2)
                x_mean = round((patches[j].x_max+patches[j+1].x_min)/2)
                line_pts = 0
                for k in range(patches[j].x_max, patches[j+1].x_min):
                    if mask[y_mean, k] > 0:
                        line_pts += 1
                # if more than two thick line points were detected, then a thick line is present in that region
                if line_pts > 2:
                    vertical_thick_lines[j] = 1
                    
            if (j < len(patches) - 9):
                # check the vertical regions between patches
                x_mean = round((patches[j].x_max+patches[j].x_min)/2)
                y_mean = round((patches[j].y_max+patches[j+9].y_min)/2)
                line_pts = 0
                for k in range(patches[j].y_max, patches[j+9].y_min):
                    if mask[k, x_mean] > 0:
                        line_pts += 1
                if line_pts > 2:
                    horizontal_thick_lines[j] = 1    
        
        vertical_thick_lines = vertical_thick_lines.reshape(9,9)
        horizontal_thick_lines = horizontal_thick_lines.reshape(9,9)
        
        ids = np.zeros((9,9),dtype=np.int8)
        
        # apply dfs and determine the id of each of the patches
        for j in range(9):
            for k in range(9):
                if ids[j,k] == 0:
                    id = np.max(ids) + 1
                    dfs(horizontal_thick_lines,vertical_thick_lines,ids,j,k,id)
        
        # save the final results
        img_no = imgs_path[i].split("/")[-1].split(".")[0]
        with open(predictions_path + str(img_no) + '_predicted.txt','w') as f:
            j = 0
            for line_isdigit, line_ids in zip(result,ids):
                line_str = ""
                for isdigit, ids in zip(line_isdigit, line_ids):
                    line_str +=  str(int(ids)) + str(isdigit)
                
                if j < 8:
                    f.write(line_str + "\n")
                else:
                    f.write(line_str)
                    
                j += 1

In [8]:
def get_matrix_boundary(matrix, bound_no):
    """
    Get the lines that are located at the boundary of the matrix in various directions
    :param matrix - 9x9 matrix from which the line is extracted
    :param bound_no - the position 0-north/1-east/2-west/3-south
    :return. the boundary line
    """ 
    if bound_no == 0:
        return matrix[0,:]
    if bound_no == 3:
        return matrix[-1,:]
    if bound_no == 1:
        return matrix[:,-1]
    if bound_no == 2:
        return matrix[:, 0]

def check_rotations(matrixes):
    """
    Match the 3 sudoku positions that define the sudoku cube.
    :param matrixes - 3 9x9 matrixes corresponding to each sudoku grid to be matched
    :return. i, j, k corresponding to the position of each sudoku after match
        i - bottom-left
        j - top
        k - bottom-right
    """ 
    first_matrix = matrixes[0,:,:]
    second_matrix = matrixes[1,:,:]
    third_matrix = matrixes[2,:,:]
    
    i = 0
    j = 0
    k = 0
    # match the first and second sudokus
    for i in range(len(matrixes)):
        found = 0
        for j in range(len(matrixes)):
            if i != j:
                first_matrix_match = get_matrix_boundary(matrixes[i], 0)
                second_matrix_match = get_matrix_boundary(matrixes[j], 3)
                if (first_matrix_match == second_matrix_match).all():
                    found = 1
                    break
        if found == 1:
            break
    
    # find the position of the third sudoku grid
    for k in range(3):
        if k != i and k != j:
            break
            
    return j,i,k
    
def warp_on_cube(img, template, position, src_pos, rotation):
    """
    Warp a sudoku grid on the sudoku cube considering the position calculated
    previously by matching the digits of each sudoku grid
    :param img - the warped image of the sudoku grid
    :param template - the cube where the grid will be pasted
    :param position - 0,1,2 corresponding to the three positions on the cube
    :param src_pos - the source position defined in pixels of the corners of the grid
    :param rotation - the rotation of the sudoku grid according to the axis
    :return. weighted image of the adding of the grid over the cube template
    """ 
    dest = []
    # statically define the destination points for the warping
    if rotation == 0:
        if position == 0:
            dest = [[295,538], [4,460], [8,157], [296,233]]
        if position == 1:
            dest = [[560,386], [295,538], [296,233], [560,82]]
        if position == 2:
            dest = [[296,233], [8,157], [272,2], [560,82]]
    else:
        if position == 0:
            dest = [[4,460], [8,157], [296,233], [295,538]]
        if position == 1:
            dest = [[295,538], [296,233], [560,82], [560,386]]
        if position == 2:
            dest = [[8,157], [272,2], [560,82], [296,233]]
        
    # define a alpha mask that will be applied over the image of the sudoku grid
    alpha_mask = np.zeros((img.shape[0],img.shape[1]))
    alpha_mask[50:350,50:350] = 255
    img = cv.cvtColor(img, cv.COLOR_BGR2BGRA) 
    img[:,:,3] = alpha_mask
    
    # get the transform and apply it to the image
    M = cv.getPerspectiveTransform(np.float32(src_pos), np.float32(dest))
    img = cv.warpPerspective(img, M, img.shape[:2], flags=cv.INTER_LINEAR)
    
    # add the BGRA warped image of the sudoku grid over the template image
    added_image = cv.addWeighted(template,1,img,1,0)
        
    return added_image

In [9]:
def evaluate_task3(train_or_test='train'):
    """
    Evaluate task 3 on the 'cube' input set
    :param train_or_test. The source of the files. It can be either 'train' or 'test'.
            To find the input images, the root folder where the notebook is located is taken
            into consideration.
    :output. The files with the resulting output will be saved at:
            <current_directory>/evaluation/submission_files/cube/"
    """
    print("Evaluate task 3")
    imgs_path = list_of_files(dirname + "/" + train_or_test + "/cube/")
    
    mean_img = np.load('mean_img_digits.npy')

    imgs_path = [path for path in imgs_path if "result" not in path and "template" not in path]
    
    predictions_path_root = "/evaluation/submission_files/"

    predictions_path = dirname + predictions_path_root + "cube/"
    
    # open the template image and transform it to a 4 channel (alpha added) image
    template = cv.imread(dirname+"/train/cube/template.jpg")
    template = cv.cvtColor(template, cv.COLOR_BGR2BGRA)
    template[:,:,3] = np.ones((template.shape[:2])) * 255
    
    for i in range(len(imgs_path)):
        print("Image " + str(i+1) + " from " + str(len(imgs_path)) + " for task 3")
        # open the image and apply an adaptiveThreshold method to find the needed features
        image = cv.imread(imgs_path[i])
        gray_image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
        blur = cv.medianBlur(gray_image, 3)
        edges = cv.adaptiveThreshold(blur,255,cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY_INV,11,3)
        
        # find the 3 biggest contors that correspind to each of the sudoku grids
        _, contours, _ = cv.findContours(edges,cv.RETR_TREE,cv.CHAIN_APPROX_SIMPLE)
        cnt = sorted(contours, key=cv.contourArea, reverse=True)
        biggest_cont = cnt[0:3]
        
        # calculate the mask for each of the grids regarding their contours
        res = []
        for big_cont in biggest_cont:
            mask = np.zeros_like(gray_image)
            cv.drawContours(mask,[big_cont],0,255,-1)
            cv.drawContours(mask,[big_cont],0,0,2)
            res.append(cv.bitwise_and(image,image,mask = mask))

        # get the corners of each of the contours
        contour_corners = []
        for contour in biggest_cont:
            box_corners = []
            rot_rect = cv.minAreaRect(contour)
            box = cv.boxPoints(rot_rect)
            box = np.int0(box)
            for p in box:
                box_corners.append([p[0],p[1]])
            contour_corners.append(box_corners)
            
        # get the rotation, the warping destination and the warped image of each of the grids
        final_dst = []
        rotation = []
        warped = []
        for j, box_corners in enumerate(contour_corners):
            if box_corners[1][1] > box_corners[3][1]:
                dst = [[350,350],[50,350],[50,50],[350,50]]
                rotation.append(0)
            else:
                dst = [[50,350],[50,50],[350,50],[350,350]]
                rotation.append(1)
    
            final_dst.append(dst)
            M = cv.getPerspectiveTransform(np.float32(box_corners), np.float32(dst))

            warped_image = cv.warpPerspective(res[j], M, res[j].shape[:2], 
                                              flags=cv.INTER_LINEAR)[:template.shape[1],:template.shape[0]]
                                                                                                  
            warped.append(warped_image)
            
        # using the difference between image and the mean images, find the digit present in the patches
        sudoku_digits = np.zeros((3,9,9))
        for n, warped_image in enumerate(warped):
            # split the sudoku grid in 81 regions that are empirically determined
            for j in range(9):
                for k in range(9): 
                    distances = []
                    # calculate the patch
                    digit = warped_image[round(300/9)*j+54:round(300/9)*(j+1)+50,
                                           round(300/9)*k+54:round(300/9)*(k+1)+50]
                    digit = cv.cvtColor(digit, cv.COLOR_BGR2GRAY)
                    
                    # calculate the euclidean distance between our patch and each of the means of the digits
                    for m in range(1,10):
                        distances.append(np.linalg.norm(digit-mean_img[m]))
                    
                    # get the digit present in the current patch
                    sudoku_digits[n,j,k] = np.argmin(distances) + 1
                    
        # match the position of each of the grids
        idx1, idx2, idx3 = check_rotations(sudoku_digits)
    
        # warp the sudokus over the cube
        t = np.zeros((template.shape), dtype=np.uint8())
        t = warp_on_cube(warped[idx1], t, 2, final_dst[0], rotation[0])
        t = warp_on_cube(warped[idx2], t, 0, final_dst[1], rotation[1])
        t = warp_on_cube(warped[idx3], t, 1, final_dst[2], rotation[2])

        # save the results to the corresponding files
        img_no = imgs_path[i].split("/")[-1].split(".")[0]
        with open(predictions_path + str(img_no) + '_predicted.txt','w') as f:
            for j, line in enumerate(sudoku_digits[idx1]):
                line_str = ""
                for digit in line:
                    line_str += str(int(digit))
                if j < 8:
                    f.write(line_str + "\n")
                else:
                    f.write(line_str + "\n\n")

            j = 0
            for line1, line2 in zip(sudoku_digits[idx2],sudoku_digits[idx3]):
                line_str1 = ""
                line_str2 = ""
                line_final = ""
                for digit1, digit2 in zip(line1, line2):
                    line_str1 += str(int(digit1))
                    line_str2 += str(int(digit2))
                line_final = line_str1 + " " + line_str2
                
                if j < 8:
                    f.write(line_final + "\n")
                else:
                    f.write(line_final)
                    
                j += 1
                
        cv.imwrite(predictions_path + str(img_no) + '_result.jpg', t)

In [10]:
evaluate_task1(train_or_test = 'train')
evaluate_task2(train_or_test = 'train')
evaluate_task3(train_or_test = 'train')

Evaluate task 1
Image 1 from 50 for task 1


/home/valentin/miniconda3/envs/virtual-env/lib/python3.7/site-packages/ipykernel_launcher.py:165: RuntimeWarning: divide by zero encountered in int_scalars


Image 2 from 50 for task 1
Image 3 from 50 for task 1
Image 4 from 50 for task 1
Image 5 from 50 for task 1
Image 6 from 50 for task 1
Image 7 from 50 for task 1
Image 8 from 50 for task 1
Image 9 from 50 for task 1
Image 10 from 50 for task 1
Image 11 from 50 for task 1
Image 12 from 50 for task 1
Image 13 from 50 for task 1
Image 14 from 50 for task 1
Image 15 from 50 for task 1
Image 16 from 50 for task 1
Image 17 from 50 for task 1
Image 18 from 50 for task 1
Image 19 from 50 for task 1
Image 20 from 50 for task 1
Image 21 from 50 for task 1
Image 22 from 50 for task 1
Image 23 from 50 for task 1
Image 24 from 50 for task 1
Image 25 from 50 for task 1
Image 26 from 50 for task 1
Image 27 from 50 for task 1
Image 28 from 50 for task 1
Image 29 from 50 for task 1
Image 30 from 50 for task 1
Image 31 from 50 for task 1
Image 32 from 50 for task 1
Image 33 from 50 for task 1
Image 34 from 50 for task 1
Image 35 from 50 for task 1
Image 36 from 50 for task 1
Image 37 from 50 for task 1
